In [1]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [2]:
env = BlackjackEnv()

In [3]:
def create_random_policy(nA):
    """
    Creates a random policy function.
    
    Args:
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities
    """
    A = np.ones(nA, dtype=float) / nA
    def policy_fn(observation):
        return A
    return policy_fn

In [11]:
def create_greedy_policy(Q):
    """
    Creates a greedy policy based on Q values.
    
    Args:
        Q: A dictionary that maps from state -> action values
        
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities.
    """
    
    def policy_fn(observation):
        return np.argmax(Q[observation])
    return policy_fn

In [15]:
def mc_control_importance_sampling(env, num_episodes, behavior_policy, discount_factor=1.0):
    """
    Monte Carlo Control Off-Policy Control using Weighted Importance Sampling.
    Finds an optimal greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        behavior_policy: The behavior to follow while generating episodes.
            A function that given an observation returns a vector of probabilities for each action.
        discount_factor: Gamma discount factor.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities. This is the optimal greedy policy.
    """
    
    # The final action-value function.
    # A dictionary that maps state -> action values
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # Our greedily policy we want to learn
    target_policy = create_greedy_policy(Q)
    C = defaultdict(float)
    
    for n in range(num_episodes):
        if (n%10000 ==0):
            print(n)
        observation = env.reset()
        episode = []
        while True:
            probability = behavior_policy(observation)
            random_action = np.random.choice(np.arange(env.action_space.n), p = probability)
            next_observation, reward, done, _ = env.step(random_action)
            episode.append((random_action,reward,observation))
            
            if done:
                break
                
            observation = next_observation
            
        inverse_episode = episode[::-1]
        G = 0
        W = 1
        
        for single_episode in inverse_episode:
            state = single_episode[2]
            reward = single_episode[1]
            action = single_episode[0]
            G = discount_factor*G + single_episode[1]
            C[(state,action)] += W
            
            Q[state][action] += (W/C[(state,action)])*(G - Q[state][action])
            
            if action != target_policy(state):
                break
                
            W = W * (1/env.action_space.n)

            
    
    # Implement this!
        
    return Q, target_policy

In [ ]:
random_policy = create_random_policy(env.action_space.n)
Q, policy = mc_control_importance_sampling(env, num_episodes=500000, behavior_policy=random_policy)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [ ]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, action_values in Q.items():
    action_value = np.max(action_values)
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")